# Basic Query with Prefer Rack
Running on 3 node cluster. Each node has namespace test with unique rack-id (1, 2, & 3) and RF=3.
So each node has all the data.  Demonstrate running SI query with Preferred Rack - runs on single node.

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

//Needed imports for SI Query
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.Operation;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

//For implementing preferred rack
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.policy.Replica;

System.out.println("Client modules imported.");

Client modules imported.


# Generate Test Data

In [4]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

In [5]:
%sh aql -c "set key_send true"

In [6]:
%sh aql -f "../aqlScripts/insert.aql"

In [7]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");
for(int i=0; i<10; i++){
    Key key = new Key("test", "testset", "key"+i);
    System.out.println("key"+i+ " : "+client.get(null, key));
}

Initialized the client and connected to the cluster.
key0 : (gen:1),(exp:453789840),(bins:(name:Sandra),(age:34))
key1 : (gen:1),(exp:453789840),(bins:(name:Jack),(age:26))
key2 : (gen:1),(exp:453789840),(bins:(name:Jill),(age:20))
key3 : (gen:1),(exp:453789840),(bins:(name:James),(age:38))
key4 : (gen:1),(exp:453789840),(bins:(name:Jim),(age:46))
key5 : (gen:1),(exp:453789840),(bins:(name:Julia),(age:62))
key6 : (gen:1),(exp:453789840),(bins:(name:Sally),(age:32))
key7 : (gen:1),(exp:453789840),(bins:(name:Sean),(age:24))
key8 : (gen:1),(exp:453789840),(bins:(name:Sam),(age:12))
key9 : (gen:1),(exp:453789840),(bins:(name:Susan),(age:42))


In [8]:
//Close the client
client.close();

In [9]:
%sh asadm --enable -e "manage sindex create numeric idx_age ns test set testset bin age"

# Run Default SI Query on all nodes of cluster.
We closed the client object after inserting records. We will create a new client object.

In [10]:
//Instantiate client object with default ClientPolicy

AerospikeClient client = new AerospikeClient("localhost", 3000);

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,30));
QueryPolicy qp = new QueryPolicy();
RecordSet rs = client.query(qp, stmt);

while(rs.next()){
    Record r = rs.getRecord();
    Key thisKey = rs.getKey();  
    System.out.println(r);
}

//Close this client
client.close();

(gen:1),(exp:453789840),(bins:(name:Sean),(age:24))
(gen:1),(exp:453789840),(bins:(name:Jill),(age:20))
(gen:1),(exp:453789840),(bins:(name:Jack),(age:26))


# Modify query to go to Preferred Rack (rack-id 1)
We will monitor the logfile output on each node.

sudo cat /var/log/aerospike/aerospike.log |grep si-query

May 14 2024 04:36:42 GMT: INFO (info): (ticker.c:885) {test} si-query: short-basic (0,0,0) long-basic (2,0,0) aggr (0,0,0) udf-bg (0,0,0) ops-bg (0,0,0) 

and see long-basic counts on each node.

In [14]:
//Instantiate client object with Preferred Rack ClientPolicy
ClientPolicy cp = new ClientPolicy();
cp.rackId = 1;
cp.rackAware = true;
AerospikeClient client = new AerospikeClient(cp, "localhost", 3000);

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,30));
QueryPolicy qp = new QueryPolicy();

//Specify query to use preferred rack
qp.replica = Replica.PREFER_RACK;

RecordSet rs = client.query(qp, stmt);

while(rs.next()){
    Record r = rs.getRecord();
    Key thisKey = rs.getKey();  
    System.out.println(r);
}

//Close this client
client.close();

(gen:1),(exp:453789840),(bins:(name:Sean),(age:24))
(gen:1),(exp:453789840),(bins:(name:Jill),(age:20))
(gen:1),(exp:453789840),(bins:(name:Jack),(age:26))


# Cleanup

In [ ]:
%sh asadm --enable -e "manage sindex delete idx_age ns test set testset"

In [ ]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"